In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm

cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
df_jo = pm.df('df_jo')
#df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = pm.sort("df_jo", ['created_at','created_at_fe']).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
#df_jo_cp = df_jo.copy()
#df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
#df_jo.info()
pd.options.display.max_columns = None
#display(df_jo)

df = df_jo 
#pm.format_to_dates(df, time_format='s') # 'min','s'
fields = ['id_cr','user_id','created_at','moderated_at','transfer_type','stat_cr' ,'amount','fee',
          'n_fees','n_backs','needs_m_check_recov', 'n_recovery', 'n_incidents', #'created_at_dow',
          'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment','type','recovery_status' 
          # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
          ]

In [4]:
# df = df.sort_values(['created_at','created_at_fe'])
# df['n_fees'] = (df['stat_cr'] == "money_back") & (df['stat_fe'] == "accepted") & (df['fee'] > 0)
# df['n_fees'] = df.groupby('user_id')['n_fees'].cumsum()


#df['created_at'] = pd.to_datetime(df['created_at'])
df = df.sort_values(by=['user_id', 'created_at'])
display(df[df['user_id'] == 47][fields]) #47 tenemos un problema con n_fees


,id_cr,user_id,created_at,moderated_at,transfer_type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check_recov,n_recovery,n_incidents,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,type,recovery_status
0,3,47,2019-11-19 13:57:53.511561,2019-11-20 12:16:50.000000,regular,canceled,1.0,0.0,1,0,1,0,1,cr_regular,0,NaT,NaT,NaN,NaT,2019-12-05 23:00:00.000000,16 days 09:02:06.488439,NaT,NaT,NaN,0,nice
53,118,47,2019-12-12 14:14:14.131479,2019-12-12 14:41:39.485394,regular,rejected,100.0,0.0,1,0,1,0,2,cr_regular,0,NaT,NaT,NaN,NaT,2019-12-19 14:14:14.131025,6 days 23:59:59.999546,NaT,NaT,NaN,0,nice
1143,1218,47,2020-04-16 08:42:15.429083,2020-04-16 09:15:24.585050,regular,money_back,100.0,0.0,1,1,0,0,2,cr_regular,0,NaT,NaT,NaN,2020-05-08 21:35:33.858794,2020-04-30 08:42:16.007000,14 days 00:00:00.577917,NaT,NaT,NaN,0,nice
1771,1832,47,2020-05-15 16:04:10.449840,2020-05-15 16:08:34.000000,regular,money_back,10.0,5.0,1,2,1,0,3,cancelled,72,2020-05-29 14:27:56.118917,2020-12-22 17:19:26.530567,Postpone Cash Request 1832,2020-08-04 18:19:04.558434,2020-09-17 22:00:00.000000,125 days 05:55:49.550160,2020-07-19 22:00:00,2020-08-03 22:00:00,before,postpone,nice
1772,1832,47,2020-05-15 16:04:10.449840,2020-05-15 16:08:34.000000,regular,money_back,10.0,5.0,1,2,1,0,4,cancelled,71,2020-05-29 14:27:56.118917,2020-12-22 17:19:34.028812,Postpone Cash Request 1832,2020-08-04 18:19:04.558434,2020-09-17 22:00:00.000000,125 days 05:55:49.550160,2020-06-19 22:00:00,2020-07-19 22:00:00,before,postpone,nice
1773,1832,47,2020-05-15 16:04:10.449840,2020-05-15 16:08:34.000000,regular,money_back,10.0,5.0,1,2,1,0,5,cancelled,47,2020-06-01 14:24:14.624776,2020-12-22 17:19:20.056527,Postpone Cash Request 1832,2020-08-04 18:19:04.558434,2020-09-17 22:00:00.000000,125 days 05:55:49.550160,2020-08-03 22:00:00,2020-08-18 22:00:00,before,postpone,nice
1774,1832,47,2020-05-15 16:04:10.449840,2020-05-15 16:08:34.000000,regular,money_back,10.0,5.0,1,2,1,0,6,cancelled,21,2020-06-23 08:20:48.513423,2020-12-22 17:19:13.611324,Postpone Cash Request 1832,2020-08-04 18:19:04.558434,2020-09-17 22:00:00.000000,125 days 05:55:49.550160,2020-06-05 22:00:00,2020-06-19 22:00:00,before,postpone,nice
1775,1832,47,2020-05-15 16:04:10.449840,2020-05-15 16:08:34.000000,regular,money_back,10.0,5.0,2,2,0,0,6,accepted,1367,2020-07-14 14:59:45.303324,2020-10-13 14:25:02.353414,Postpone Cash Request 1832,2020-08-04 18:19:04.558434,2020-09-17 22:00:00.000000,125 days 05:55:49.550160,2020-08-18 22:00:00,2020-09-17 22:00:00,before,postpone,nice
11530,10942,47,2020-08-05 09:34:16.818322,2020-08-05 09:35:50.000000,instant,money_back,10.0,5.0,3,3,0,0,6,accepted,2792,2020-08-05 09:35:51.945346,2021-01-21 15:50:35.089962,Instant Payment Cash Request 10942,2020-09-22 15:05:36.000000,2020-09-29 22:00:00.000000,55 days 12:25:43.181678,NaT,NaT,after,instant_payment,nice
11531,10942,47,2020-08-05 09:34:16.818322,2020-08-05 09:35:50.000000,instant,money_back,10.0,5.0,4,3,0,0,6,accepted,5609,2020-09-01 10:54:11.067922,2020-10-13 14:25:08.138501,Postpone Cash Request 10942,2020-09-22 15:05:36.000000,2020-09-29 22:00:00.000000,55 days 12:25:43.181678,2020-09-04 22:00:00,2020-09-29 22:00:00,before,postpone,nice


In [8]:
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.sort_values(by=['user_id', 'created_at'])

# obtenemos la primera fila de cada 'user_id' (la más antigua)
primer_registro = df.groupby('user_id').first()
primer_registro['no_moderat'] = primer_registro['moderated_at'].isnull()
no_moderats = primer_registro.reset_index() #level='user_id')
no_moderats = primer_registro[primer_registro['no_moderat'] == True]
#resultat = no_moderats.reset_index()[['id_cr','id_fe','user_id','created_at','moderated_at','','no_moderat','transfer_type']]
resultat = no_moderats[no_moderats.transfer_type.isin(['regular','instant'])].reset_index()
#display(resultat.sort_values(['created_at']))
#display(resultat[resultat.user_id > 0 ][fields].sort_values(['created_at','created_at_fe']).reset_index(drop=True))
#display(resultat[fields].sort_values('created_at').head(4))
#resultat.user_id.count()

df['moderada'] = df['moderated_at'].apply(lambda x: 1 if not pd.isnull(x) else 0)
cnt = df[df['moderada'] == 1]
print(cnt.moderada.count())

cnt = df[df['moderada'] == 0]
print(cnt.moderada.count())

display(df[df['user_id'] == 47].sort_values('created_at')[fields])




21757
10335


,id_cr,user_id,created_at,moderated_at,transfer_type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check_recov,n_recovery,n_incidents,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,type,recovery_status
0,3,47,2019-11-19 13:57:53,2019-11-20 12:16:50,regular,canceled,1.0,0.0,6,0,1,0,1,cr_regular,0,NaT,NaT,NaN,NaT,2019-12-05 23:00:00,16 days 09:02:06.488439,NaT,NaT,NaN,0,nice
53,118,47,2019-12-12 14:14:14,2019-12-12 14:41:39,regular,rejected,100.0,0.0,6,0,1,0,2,cr_regular,0,NaT,NaT,NaN,NaT,2019-12-19 14:14:14,6 days 23:59:59.999546,NaT,NaT,NaN,0,nice
1143,1218,47,2020-04-16 08:42:15,2020-04-16 09:15:24,regular,money_back,100.0,0.0,6,1,0,0,2,cr_regular,0,NaT,NaT,NaN,2020-05-08 21:35:33,2020-04-30 08:42:16,14 days 00:00:00.577917,NaT,NaT,NaN,0,nice
1771,1832,47,2020-05-15 16:04:10,2020-05-15 16:08:34,regular,money_back,10.0,5.0,0,2,1,0,3,cancelled,71,2020-05-29 14:27:56,2020-12-22 17:19:34,Postpone Cash Request 1832,2020-08-04 18:19:04,2020-09-17 22:00:00,125 days 05:55:49.550160,2020-06-19 22:00:00,2020-07-19 22:00:00,before,postpone,nice
1772,1832,47,2020-05-15 16:04:10,2020-05-15 16:08:34,regular,money_back,10.0,5.0,0,2,1,0,4,cancelled,72,2020-05-29 14:27:56,2020-12-22 17:19:26,Postpone Cash Request 1832,2020-08-04 18:19:04,2020-09-17 22:00:00,125 days 05:55:49.550160,2020-07-19 22:00:00,2020-08-03 22:00:00,before,postpone,nice
1773,1832,47,2020-05-15 16:04:10,2020-05-15 16:08:34,regular,money_back,10.0,5.0,0,2,1,0,5,cancelled,47,2020-06-01 14:24:14,2020-12-22 17:19:20,Postpone Cash Request 1832,2020-08-04 18:19:04,2020-09-17 22:00:00,125 days 05:55:49.550160,2020-08-03 22:00:00,2020-08-18 22:00:00,before,postpone,nice
1774,1832,47,2020-05-15 16:04:10,2020-05-15 16:08:34,regular,money_back,10.0,5.0,0,2,1,0,6,cancelled,21,2020-06-23 08:20:48,2020-12-22 17:19:13,Postpone Cash Request 1832,2020-08-04 18:19:04,2020-09-17 22:00:00,125 days 05:55:49.550160,2020-06-05 22:00:00,2020-06-19 22:00:00,before,postpone,nice
1775,1832,47,2020-05-15 16:04:10,2020-05-15 16:08:34,regular,money_back,10.0,5.0,1,2,0,0,6,accepted,1367,2020-07-14 14:59:45,2020-10-13 14:25:02,Postpone Cash Request 1832,2020-08-04 18:19:04,2020-09-17 22:00:00,125 days 05:55:49.550160,2020-08-18 22:00:00,2020-09-17 22:00:00,before,postpone,nice
11530,10942,47,2020-08-05 09:34:16,2020-08-05 09:35:50,instant,money_back,10.0,5.0,2,3,0,0,6,accepted,2792,2020-08-05 09:35:51,2021-01-21 15:50:35,Instant Payment Cash Request 10942,2020-09-22 15:05:36,2020-09-29 22:00:00,55 days 12:25:43.181678,NaT,NaT,after,instant_payment,nice
11531,10942,47,2020-08-05 09:34:16,2020-08-05 09:35:50,instant,money_back,10.0,5.0,3,3,0,0,6,accepted,5609,2020-09-01 10:54:11,2020-10-13 14:25:08,Postpone Cash Request 10942,2020-09-22 15:05:36,2020-09-29 22:00:00,55 days 12:25:43.181678,2020-09-04 22:00:00,2020-09-29 22:00:00,before,postpone,nice
